# Miscellaneous from Youtube, Google, ... etc

## Generator : Performance

generator는 변수를 메모리에 저장하지 않고 바로 리턴하기 때문에  
메모리, 시간 면에서 return으로 데이터를 반환하는 것보다 효율적이다.  

파이썬 3.8에서 time.clock()은 사라질 것이니 process_time, perf_counter 등을 사용하라고 한다.

In [1]:
import random, time
import memory_profiler as mem_profile

In [2]:
names = ['John', 'Corey', 'Adam', 'Steve', 'Rick', 'Thomas']
majors = ['Math', 'Engineering', 'CompSci', 'Arts', 'Business']

In [3]:
print("Memory (Before): {}Mb".format(mem_profile.memory_usage()))

Memory (Before): [46.08203125]Mb


In [4]:
def people_list(num_people):
    result = []
    for i in range(num_people):
        person = {
            'id': i,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        result.append(person)
    return result

In [5]:
def people_generator(num_people):
    for i in range(num_people):
        person = {
            'id': i,
            'name': random.choice(names),
            'major': random.choice(majors)
        }
        yield person

In [6]:
t1 = time.process_time()
people = people_list(1000000)
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [339.546875]Mb
Took 1.399382 Seconds


In [7]:
t1 = time.process_time()
people = people_generator(1000000)
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [50.08203125]Mb
Took 0.09962199999999988 Seconds


In [8]:
t1 = time.process_time()
people = list(people_generator(1000000))
t2 = time.process_time()

print("Memory (After): {}Mb".format(mem_profile.memory_usage()))
print("Took {} Seconds".format(t2-t1))

Memory (After): [341.9765625]Mb
Took 1.4080149999999998 Seconds


<hr>

## Variable Scope

**LEGB**  
**L**ocal (variables defined in functions)  
**E**nclosing (variables defined in the local scope of enclosing functions)  
**G**lobal (variables defined at the top level of a module or global keyword)  
**B**uilt-in (variables pre assigned in Python)

위의 순서 = 파이썬이 변수의 영역을 체크하는 순서

#### global vs. local

In [9]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    print(y)
    
test()

local y


test() 함수에서 x를 출력하는 것 : global 변수 출력  
First, find local scope  

In [10]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    print(x) # global x
    
test()

global x


In [11]:
# main body 
x = 'global x'

def test():
    # y is local to test function
    y = 'local y'
    
test()
# y 변수는 test() 함수 바깥에 존재하지 않음 (Error)
print(y)

NameError: name 'y' is not defined

test() 함수에서 x를 over-write 하지 않는다.

In [13]:
x = 'global x'

def test():
    x = 'local x'
    print(x)
    
test()
print(x)

local x
global x


global 키워드 : global scope의 변수를 사용함

In [14]:
x = 'global x'

def test():
    global x
    x = 'local x'
    print(x)
    
test()
print(x)

local x
local x


global 영역에 존재하지 않는 변수는 생성해서 에러 발생 안 함

In [15]:
# x = 'global x'

def test():
    global x
    x = 'local x'
    print(x)
    
test()
print(x)

local x
local x


global 키워드를 자주 사용하지 않는다.  
만약 자주 사용한다면, 함수 외부에서 오버라이딩 되는 것을 매우 고려해야함

In [16]:
# z 변수도 local scope 에 존재함
# 차이점은 함수에 값을 전달하는 함수 파라미터라는 것
def test(z):
    x = 'local x'
    print(z)
    
test('local z')

local z


In [17]:
def test(z):
    x = 'local x'
    print(z)
    
test('local z')
# z is only local to test function
print(z)

local z


NameError: name 'z' is not defined

#### built-in scope  
python에 미리 할당된 이름들

In [18]:
m = min([5, 1, 4, 3, 2]) # built-in function
print(m)

1


In [19]:
import builtins

# dir() : 객체의 속성들의 리스트
print(dir(builtins))

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

내장 함수, 변수는 오버라이딩이 가능함  
built-in scope보다 global scope에서 먼저 변수, 함수를 찾기 때문에 가능함  
따라서 주의해라

In [20]:
def min():
    pass

In [21]:
min([2, 1, 3])

TypeError: min() takes 0 positional arguments but 1 was given

#### enclosing

nested function

In [22]:
def outer():
    # local x to outer func
    x = 'outer x'
    
    def inner():
        # local x to inner func
        x = 'inner x'
        print(x)  # inner x
    
    inner()
    print(x)  # outer x
    
outer()

inner x
outer x


enclosing scope  
= enclosing function의 local scope

In [23]:
def outer():
    # local x to outer func
    x = 'outer x'
    
    def inner():
        # check local x -> doesn't exist
        # check enclosing x -> exists
        # inner's enclosing func. is outer func.
        print(x)  # outer x
    
    inner()
    print(x)  # outer x
    
outer()

outer x
outer x


In [25]:
def outer():
    
    def inner():
        a = 'inner a'
        print(a)  # inner a
    
    inner()
    print(a)  # Error
    
outer()

inner a


NameError: name 'a' is not defined

enclosing scope의 변수를 변경하려면?  
nonlocal 키워드  

global 변수보다 자주 사용된다  
클로저, 데코레이터의 상태를 변경하기 위해

In [26]:
# inner func.에서 enclosing x 를 변경하려면?
def outer():
    x = 'outer x'
    
    def inner():
        nonlocal x
        x = 'inner x'  # overwrite
        print(x)  # inner x
    
    inner()
    print(x)  # inner x
    
outer()

inner x
inner x


In [27]:
x = 'global x'

def outer():
    x = 'outer x'
    
    def inner():
        x = 'inner x'
        print(x)
        
    inner()
    print(x)

outer()
print(x)

inner x
outer x
global x


<hr>

## First-Class Functions

Wikipedia에 의한 First-Class Functions :  
first-class citizen으로 함수를 다루는 것(treat)

First-Class Citizen (Programming) :  
an entity which supports all the operations(being passed as an argument, returned from a function, and assigned to a variable) generally available to other entities  
(함수의 매개변수로 전달 가능하고, 함수의 반환값으로 반환될 수 있고, 어떤 변수에 할당가능한 개체)

함수를 객체나 변수처럼 다룰 수 있다

In [28]:
def square(x):
    return x * x

f = square(5)

print(square)
print(f)

<function square at 0x7f93ae2e0170>
25


함수명에 () 소괄호를 붙이는 것 = 함수를 실행한다는 의미

In [30]:
f = square # 실행 X, 함수자체를 의미

In [31]:
print(square)
print(f)

<function square at 0x7f93ae2e0170>
<function square at 0x7f93ae2e0170>


In [32]:
print(f(5))

25


대표적으로 map 함수  
함수를 인자로 전달받는다.

In [33]:
def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i))
    return result

squares = my_map(square, [1, 2, 3, 4, 5])

print(squares)

[1, 4, 9, 16, 25]


In [34]:
def cube(x):
    return x * x * x

cubes = my_map(cube, [1, 2, 3, 4, 5])

print(cubes)

[1, 8, 27, 64, 125]


함수를 리턴하는 간단한 예제

In [35]:
def logger(msg):
    
    def log_message():
        print('Log:', msg)
        
    return log_message

log_hi = logger('Hi!')  # log_message 함수
log_hi()

Log: Hi!


practical에 가까운 example

In [37]:
def html_tag(tag):
    
    def wrap_text(msg):
        print("<{0}>{1}</{0}>".format(tag, msg))
        
    return wrap_text

print_h1 = html_tag('h1')
print_h1('Test Headline')
print_h1('Another Headline')

print_p = html_tag('p')
print_p('Test Paragraph')

<h1>Test Headline</h1>
<h1>Another Headline</h1>
<p>Test Paragraph</p>


*Ref. https://youtu.be/kr0mpwqttM0*

<hr>

## Closures

Wikipedia 에 의한 Closure :  
A record storing a function together with an environment  

In [38]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        # message : free variable (inner_func에 정의되어 있지 않지만 접근 가능한 변수)
        print(message)
        
    return inner_func()

outer_func()

Hi


In [39]:
def outer_func():
    message = 'Hi'
    
    def inner_func():
        print(message)
        
    return inner_func

my_func = outer_func()

print(my_func)
print(my_func.__name__)

<function outer_func.<locals>.inner_func at 0x7f93ae76b9e0>
inner_func


In [40]:
"""
easy/simple meaning of closure
: can access to variables in the local scope in which it was created
"""
my_func()
my_func()
my_func()

Hi
Hi
Hi


closure closes over the free variables from their environment  
message is a free variable

In [41]:
def outer_func(msg):
    message = msg
    
    def inner_func():
        print(message)
        
    return inner_func

hi_func = outer_func('Hi')
hello_func = outer_func('Hello')

In [42]:
hi_func()

Hi


In [43]:
hello_func()

Hello


In [44]:
import logging

logging.basicConfig(filename='example.log', level=logging.INFO)

def logger(func):
    def log_func(*args):
        logging.info('Running "{}" with arguments {}'.format(func.__name__, args))
        print(func(*args))
    return log_func

def add(x, y):
    return x+y

def sub(x, y):
    return x-y

add_logger = logger(add)
sub_logger = logger(sub)

add_logger(3, 3)
add_logger(4, 5)

sub_logger(10, 5)
sub_logger(20, 10)

6
9
5
10


*Ref. https://youtu.be/swU3c34d2NQ*

<hr>

## Decorators

매개변수로 함수를 전달받는 함수  

In [1]:
def outer_function():
    message = 'Hi'  # free var.
    
    def inner_function():
        print(message)
    return inner_function()

outer_function()

Hi


In [3]:
def outer_function():
    message = 'Hi'  # free var.
    
    def inner_function():
        print(message)
    return inner_function

my_func = outer_function()
my_func()
my_func()
my_func()

Hi
Hi
Hi


In [4]:
def outer_function(msg):
    message = msg  # free var.
    
    def inner_function():
        print(message)
    return inner_function

hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

In [5]:
hi_func()
bye_func()

Hi
Bye


##### decorator

In [7]:
def decorator_function(org_func):
    def wrapper_function():
        org_func()
    return wrapper_function

def display():
    print('display function ran')
    
decorated_display = decorator_function(display)

decorated_display()

display function ran


In [8]:
def decorator_function(org_func):
    def wrapper_function():
        return org_func()
    return wrapper_function

def display():
    print('display function ran')
    
decorated_display = decorator_function(display)

decorated_display()

display function ran


In [9]:
def decorator_function(org_func):
    def wrapper_function():
        print('wrapper executed this before {}'.format(org_func.__name__))
        return org_func()
    return wrapper_function

# display 함수를 decorator_function(display) 로 wrapping 
@decorator_function
def display():
    print('display function ran')
    
display()

wrapper executed this before display
display function ran


In [10]:
def decorator_function(org_func):
    def wrapper_function(*args, **kwargs):  # can accept arguments
        print('wrapper executed this before {}'.format(org_func.__name__))
        return org_func(*args, **kwargs)
    return wrapper_function

@decorator_function
def display():
    print('display function ran')

@decorator_function
def display_info(name, age):
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)
display()

wrapper executed this before display_info
display_info ran with (An, 24)
wrapper executed this before display
display function ran


In [12]:
# decorator class
class decorator_class(object):
    
    def __init__(self, org_func):
        self.org_func = org_func
        
    # wrapper function
    def __call__(self, *args, **kwargs):
        print('call method executed this before {}'.format(self.org_func.__name__))
        return self.org_func(*args, **kwargs)
    
    
@decorator_class
def display():
    print('display function ran')

@decorator_class
def display_info(name, age):
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)
display()

call method executed this before display_info
display_info ran with (An, 24)
call method executed this before display
display function ran


practical example

In [13]:
def my_logger(org_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(org_func.__name__), level=logging.INFO)
    
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs)
        )
        return org_func(*args, **kwargs)
    
    return wrapper

def my_timer(org_func):
    import time
    
    def wrapper(*args, **kwargs):
        t1 = time.time()
        ret = org_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in : {} sec'.format(org_func.__name__, t2))
        
    return wrapper

@my_logger
def display_info(name, age):
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)

display_info ran with (An, 24)


In [14]:
display_info('John', 26)

display_info ran with (John, 26)


In [15]:
import time

@my_timer
def display_info(name, age):
    time.sleep(1)
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)

display_info ran with (An, 24)
display_info ran in : 1.0026600360870361 sec


In [16]:
import time

@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)

display_info ran with (An, 24)
wrapper ran in : 1.0032761096954346 sec


In [17]:
import time

# display_info = my_logger(my_timer(display_info))

@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('An', 24)

display_info ran with (An, 24)
display_info ran in : 1.000396966934204 sec


In [18]:
from functools import wraps

def my_logger(org_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(org_func.__name__), level=logging.INFO)
    
    @wraps(org_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs)
        )
        return org_func(*args, **kwargs)
    
    return wrapper

def my_timer(org_func):
    import time
    
    @wraps(org_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        ret = org_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in : {} sec'.format(org_func.__name__, t2))
        
    return wrapper

@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print('display_info ran with ({}, {})'.format(name, age))
    
display_info('Emily', 21)

display_info ran with (Emily, 21)
display_info ran in : 1.000288963317871 sec


<hr>

## Duck Typing and Easier to Ask Forgiveness than Permission (EAFP)

What is pythonic?  
- clean and readable 코드를 작성하기 위해 python convention을 따르는 것  

Duck typing  
- EAFP  
- 명칭의 유래? 오리처럼 걷고 꽥하는 객체를 가정  
- 왜 더 좋은가? 발생가능한 아주 많은 예외상황에 대해 조금 더 빠르다(ask permission = 객체에 몇 차례 더 접근해야함) + 더 가독성이 좋음   
- race condition을 피할 수 있음  

In [1]:
class Duck:
    
    def quack(self):
        print('Quack, quack')
        
    def fly(self):
        print('Flap, Flap!')

In [2]:
class Person:
    
    def quack(self):
        print("I'm Quacking Like a Duck !")
        
    def fly(self):
        print("I'm Flapping my Arms !")

In [3]:
def quack_and_fly(thing):
    # Not Duck-Typed (Non-Pythonic)
    if isinstance(thing, Duck):
        thing.quack()
        thing.fly()
    else:
        print('This has to be a Duck!')

In [4]:
d = Duck()
quack_and_fly(d)

Quack, quack
Flap, Flap!


In [5]:
p = Person()
quack_and_fly(p)

This has to be a Duck!


메서드를 가지고 있다면 호출이 되게 하라 (인스턴스 타입 확인하지 않고)

In [6]:
def quack_and_fly(thing):
    thing.quack()
    thing.fly()

In [7]:
d = Duck()
quack_and_fly(d)

Quack, quack
Flap, Flap!


In [8]:
p = Person()
quack_and_fly(p)

I'm Quacking Like a Duck !
I'm Flapping my Arms !


LBYL (Non-Pythonic)

In [9]:
def quack_and_fly(thing):
    if hasattr(thing, 'quack'):  # ask permission
        if callable(thing.quack):
            thing.quack()
            
    if hasattr(thing, 'fly'):   # ask permission
        if callable(thing.fly):
            thing.fly()

In [10]:
d = Duck()
quack_and_fly(d)

Quack, quack
Flap, Flap!


In [11]:
p = Person()
quack_and_fly(p)

I'm Quacking Like a Duck !
I'm Flapping my Arms !


EAFP (Pythonic)

In [15]:
def quack_and_fly(thing):
    try:
        thing.quack()
        thing.fly()
        thing.bark()
    except AttributeError as e:
        print(e)

In [16]:
d = Duck()
quack_and_fly(d)

Quack, quack
Flap, Flap!
'Duck' object has no attribute 'bark'


In [17]:
p = Person()
quack_and_fly(p)

I'm Quacking Like a Duck !
I'm Flapping my Arms !
'Person' object has no attribute 'bark'


Another Example

In [18]:
person = {'name': 'Jess', 'age': 23, 'job': 'Programmer'}

In [19]:
# LBYL (Non-Pythonic)
if 'name' in person and 'age' in person and 'job' in person:
    print("I'm {name}. I'm {age} years old and I am a {job}".format(**person))
else:
    print('Missing some keys')

I'm Jess. I'm 23 years old and I am a Programmer


In [20]:
# EAFP (Pythonic)
try:
    print("I'm {name}. I'm {age} years old and I am a {job}".format(**person))
except KeyError as e:
    print("Missig {} key".format(e))

I'm Jess. I'm 23 years old and I am a Programmer


In [21]:
person = {'name': 'Jess', 'age': 23}

In [22]:
# LBYL (Non-Pythonic)
if 'name' in person and 'age' in person and 'job' in person:
    print("I'm {name}. I'm {age} years old and I am a {job}".format(**person))
else:
    print('Missing some keys')

Missing some keys


In [23]:
# EAFP (Pythonic)
try:
    print("I'm {name}. I'm {age} years old and I am a {job}".format(**person))
except KeyError as e:
    print("Missig {} key".format(e))

Missig 'job' key


python docs example (race condition case)

In [24]:
import os
my_file = 'test.txt'

In [25]:
# non-pythonic
if os.access(my_file, os.R_OK):
    with open(my_file) as f:  # race condition 발생 가능 (여기서 readable 하지 않을 수 있음)
        print(f.read())
else:
    print('File can not be accessed')

this is new file!
 first file example!


In [27]:
# pythonic
try:
    f = open(my_file)
    print(f.read())
except IOError as e:
    print('File can not be accessed')

this is new file!
 first file example!


결론
- 실행 전에 여러 조건을 검사하는 것보다  
- 일단 try 하고 되면 good, 안 되면 execption 으로 처리하는 것이  
- 좀 더 pythonic 한 코드 !

<hr>